In [49]:
from pymol import cmd

# Mapping of one-letter to three-letter amino acid codes
amino_acid_map = {
    'A': 'ALA',  # Alanine
    'R': 'ARG',  # Arginine
    'N': 'ASN',  # Asparagine
    'D': 'ASP',  # Aspartic Acid
    'C': 'CYS',  # Cysteine
    'E': 'GLU',  # Glutamic Acid
    'Q': 'GLN',  # Glutamine
    'G': 'GLY',  # Glycine
    'H': 'HIS',  # Histidine
    'I': 'ILE',  # Isoleucine
    'L': 'LEU',  # Leucine
    'K': 'LYS',  # Lysine
    'M': 'MET',  # Methionine
    'F': 'PHE',  # Phenylalanine
    'P': 'PRO',  # Proline
    'S': 'SER',  # Serine
    'T': 'THR',  # Threonine
    'W': 'TRP',  # Tryptophan
    'Y': 'TYR',  # Tyrosine
    'V': 'VAL',  # Valine
}

# Function to mutate a residue in a given PDB file
def mutate_residues(pdb_file, mutations, output_dir="output_directory"):
    """
    Mutate residues in a PDB file using PyMOL.
    
    Parameters:
    - pdb_file: Path to the input PDB file.
    - mutations: List of mutation tuples, e.g., [(4, "S", "R")].
      Each tuple should be in the form (residue_number, original_one_letter, mutated_one_letter).
    - output_dir: Directory to save the modified PDB file.
    """
    # Load the PDB file
    cmd.load(pdb_file, "structure")
    
    # Loop through each mutation request
    for resi, original_residue, target_residue in mutations:
        # Mutation will always apply to chain X in the structure
        selection = f"chain X and resi {resi}"
        
        # Apply mutation using PyMOL's mutagenesis wizard
        cmd.wizard("mutagenesis")
        cmd.do(f"select {selection}")
        cmd.get_wizard().do_select(selection)
        
        # Set the target residue for mutation using three-letter code
        if target_residue == "H":  # Check if the target residue is Histidine
            three_letter_residue = "HID"  # Use HID for the imidazole form of histidine
        else:
            three_letter_residue = amino_acid_map[target_residue]
        
        cmd.get_wizard().set_mode(three_letter_residue)
        cmd.get_wizard().apply()
        
        # Use alter command to ensure correct naming with full amino acid names
        cmd.alter(selection, f"resn='{three_letter_residue}'")  # Set residue name to full amino acid name
        cmd.alter(selection, "name=name")  # Reassign correct atom names
        
        # Close the mutagenesis wizard
        cmd.set_wizard()

    # Remove hydrogens after mutations
    cmd.remove("hydro")
    
    # Generate output file name using original and mutated one-letter codes
    output_file = f"{output_dir}/{original_residue}{resi}{target_residue}.pdb"
    
    # Save the mutated structure to a new PDB file
    cmd.save(output_file, "structure", format="pdb")
    
    # Clear all after processing
    cmd.delete("all")
    print(f"Mutations applied. Saved to {output_file}.")

# Usage example
pdb_file_path = "/home/hp/nayanika/github/GPX6/prep_structures/original_humansec.pdb"
mutations_list = [
    (195, "N", "S"),   # This will apply mutation to chain X
    # Add more mutations as needed
]
output_dir = "/home/hp/nayanika/github/GPX6/prep_structures/humansec"

# Run the mutation function
mutate_residues(pdb_file_path, mutations_list, output_dir)


PyMOL>select chain X and resi 195
Selected!
 Selector: selection "sele" defined with 14 atoms.
 ExecutiveRMSPairs: RMSD =    0.043 (4 to 4 atoms)
 Mutagenesis: 3 rotamers loaded.
 Rotamer 1/3, strain=25.43
 ExecutiveRMSPairs: RMSD =    0.043 (4 to 4 atoms)
 Mutagenesis: 2 rotamers loaded.
 Rotamer 2/2, strain=22.85
Mutations applied. Saved to /home/hp/nayanika/github/GPX6/prep_structures/humansec/N195S.pdb.


In [15]:
from pymol import cmd

# Mapping of one-letter to three-letter amino acid codes
amino_acid_map = {
    'A': 'ALA', 'R': 'ARG', 'N': 'ASN', 'D': 'ASP', 'C': 'CYS', 'E': 'GLU', 
    'Q': 'GLN', 'G': 'GLY', 'H': 'HIS', 'I': 'ILE', 'L': 'LEU', 'K': 'LYS', 
    'M': 'MET', 'F': 'PHE', 'P': 'PRO', 'S': 'SER', 'T': 'THR', 'W': 'TRP', 
    'Y': 'TYR', 'V': 'VAL', 'U': 'SEC'  # Adding SEC (Selenocysteine)
}

# Function to mutate a residue in a given PDB file
def mutate_residues(pdb_file, mutations, output_dir="output_directory"):
    """
    Mutate residues in a PDB file using PyMOL.
    
    Parameters:
    - pdb_file: Path to the input PDB file.
    - mutations: List of mutation tuples, e.g., [(49, "S", "U")].
      Each tuple should be in the form (residue_number, original_one_letter, mutated_one_letter).
    - output_dir: Directory to save the modified PDB file.
    """
    # Load the PDB file
    cmd.load(pdb_file, "structure")
    
    # Loop through each mutation request
    for resi, original_residue, target_residue in mutations:
        selection = f"chain X and resi {resi}"  # Apply mutation to chain X
        
        # Use mutagenesis wizard for mutation
        cmd.wizard("mutagenesis")
        cmd.do(f"select {selection}")
        cmd.get_wizard().do_select(selection)
        
        # Handle SEC residue specifically with unique atom names
        if target_residue == "U":
            three_letter_residue = "SEC"  # Use SEC for selenocysteine
            # Alter to add SEC-specific atom names
            cmd.alter(selection, f"resn='{three_letter_residue}'")
            cmd.alter(selection, "name='N' if name == ' N  ' else name") # Example
            cmd.alter(selection, "name='SE' if name == ' SE ' else name") # SEC atom SE
            cmd.alter(selection, "name='HG1' if name == 'HG1' else name") # SEC hydrogen
        
        elif target_residue == "H":
            three_letter_residue = "HID"  # Use HID for histidine's imidazole form
        else:
            three_letter_residue = amino_acid_map.get(target_residue, "UNK")
        
        cmd.get_wizard().set_mode(three_letter_residue)
        cmd.get_wizard().apply()
        
        # Ensure the correct naming for all atoms
        cmd.alter(selection, f"resn='{three_letter_residue}'")
        
        # Close the mutagenesis wizard
        cmd.set_wizard()
    
    # Remove all hydrogens if needed
    cmd.remove("hydro")
    
    # Output file name using mutation details
    output_file = f"{output_dir}/{original_residue}{resi}{target_residue}.pdb"
    
    # Save the mutated structure
    cmd.save(output_file, "structure", format="pdb")
    cmd.delete("all")
    print(f"Mutations applied. Saved to {output_file}.")

# Example usage
pdb_file_path = "/home/hp/nayanika/github/GPX6/prep_structures/original_humansec.pdb"
mutations_list = [
    (33, "K", "Q"),  # Example mutation to SEC
]
output_dir = "/home/hp/nayanika/github/GPX6/prep_structures/humansec"

# Run the mutation function
mutate_residues(pdb_file_path, mutations_list, output_dir)


PyMOL>select chain X and resi 33
Selected!
 Selector: selection "sele" defined with 22 atoms.
 ExecutiveRMSPairs: RMSD =    0.051 (4 to 4 atoms)
 Mutagenesis: 21 rotamers loaded.
 Rotamer 3/21, strain=20.02
 ExecutiveRMSPairs: RMSD =    0.051 (4 to 4 atoms)
 Mutagenesis: 21 rotamers loaded.
 Rotamer 3/21, strain=20.02
Mutations applied. Saved to /home/hp/nayanika/github/GPX6/prep_structures/humansec/K33Q.pdb.


In [11]:
import os
from pymol import cmd

# Mapping of one-letter to three-letter amino acid codes
amino_acid_map = {
    'A': 'ALA', 'R': 'ARG', 'N': 'ASN', 'D': 'ASP', 'C': 'CYS', 'E': 'GLU', 
    'Q': 'GLN', 'G': 'GLY', 'H': 'HIS', 'I': 'ILE', 'L': 'LEU', 'K': 'LYS', 
    'M': 'MET', 'F': 'PHE', 'P': 'PRO', 'S': 'SER', 'T': 'THR', 'W': 'TRP', 
    'Y': 'TYR', 'V': 'VAL', 'U': 'SEC'  # Including SEC (Selenocysteine)
}

# Mutation list based on the table provided (Human -> Mouse)
mutations_list = [
    (3, 'N', 'K'), (4, 'R', 'S'), (16, 'I', 'V'), (22, 'L', 'N'), (24, 'L', 'I'),
    (25, 'N', 'D'), (27, 'E', 'G'), (29, 'Y', 'F'), (30, 'I', 'V'), (31, 'Q', 'N'),
    (33, 'K', 'Q'), (35, 'F', 'Y'), (40, 'V', 'I'), (47, 'A', 'S'), (48, 'Y', 'F'),
    (49, 'U', 'C'), (52, 'A', 'T'), (54, 'Q', 'T'), (60, 'A', 'T'), (67, 'N', 'P'),
    (69, 'G', 'N'), (71, 'I', 'T'), (74, 'A', 'G'), (87, 'T', 'K'), (99, 'C', 'R'),
    (102, 'S', 'G'), (104, 'F', 'Y'), (107, 'S', 'N'), (119, 'E', 'D'), (120, 'K', 'N'),
    (126, 'T', 'S'), (137, 'D', 'E'), (139, 'L', 'F'), (142, 'S', 'P'), (143, 'S', 'E'),
    (144, 'Q', 'H'), (148, 'E', 'D'), (173, 'H', 'R'), (177, 'Q', 'H'), (178, 'A', 'T'),
    (181, 'S', 'R'), (182, 'T', 'I'), (184, 'K', 'Q'), (188, 'L', 'M'), (192, 'K', 'N'),
    (194, 'F', 'T'), (195, 'N', 'S')
]

# Directory to save the modified structures
output_dir = "/home/hp/nayanika/github/GPX6/prep_structures/humansec"

# Ensure the output directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Function to mutate residues in a given PDB file
def mutate_residues(pdb_file, mutations):
    """
    Mutate residues in a PDB file using PyMOL and save each mutation as a separate file.
    
    Parameters:
    - pdb_file: Path to the input PDB file.
    - mutations: List of mutation tuples, e.g., [(49, "S", "U")].
    """
    # Load the PDB file
    cmd.load(pdb_file, "structure")
    
    # Loop through each mutation request
    for resi, original_residue, target_residue in mutations:
        selection = f"chain X and resi {resi}"  # Apply mutation to chain X
        
        # Get the current residue at the position using iterate
        current_residue = None
        cmd.iterate(selection, 'current_residue = resn')
        
        # Check if the current residue is the same as the target residue
        if current_residue == target_residue:
            print(f"Residue {resi} is already {target_residue}. Skipping mutation.")
            continue  # Skip the mutation if the residue is already the target
        
        # Use mutagenesis wizard for mutation
        cmd.wizard("mutagenesis")
        cmd.do(f"select {selection}")
        cmd.get_wizard().do_select(selection)
        
        # Special handling for SEC at position 49
        if resi == 49 and target_residue == 'U':  # We specifically mutate CYS to SEC
            three_letter_residue = "SEC"
            # For SEC, the atoms HB3, HB2, and others must be handled
            cmd.alter(f"resi {resi} and chain X", "resn='SEC'")
            # Add custom atom names or handle SEC-specific atoms if necessary
            cmd.alter(f"resi {resi} and chain X", "name='HB3' if name == 'HB2' else name")
            cmd.alter(f"resi {resi} and chain X", "name='HG1' if name == 'HG' else name")
            cmd.alter(f"resi {resi} and chain X", "name='HN' if name == 'H' else name")
        else:
            three_letter_residue = amino_acid_map.get(target_residue, "UNK")
            cmd.get_wizard().set_mode(three_letter_residue)
            cmd.get_wizard().apply()
        
        # Close the mutagenesis wizard
        cmd.set_wizard()
        
        # Remove hydrogens
        cmd.remove("hydro")
        
        # Output file name using residue name and number (e.g., "N3K.pdb" for N->K at residue 3)
        output_file = f"{output_dir}/{original_residue}{resi}{target_residue}.pdb"
        
        # Save the mutated structure
        cmd.save(output_file, "structure", format="pdb")
        print(f"Mutation applied at residue {resi}: {original_residue} -> {three_letter_residue}. Saved to {output_file}.")
    
    cmd.delete("all")

# Example usage
pdb_file_path = "/home/hp/nayanika/github/GPX6/prep_structures/original_humansec.pdb"

# Run the mutation function
mutate_residues(pdb_file_path, mutations_list)


PyMOL>select chain X and resi 3
Selected!
 Selector: selection "sele" defined with 55 atoms.
Mutation applied at residue 3: N -> LYS. Saved to /home/hp/nayanika/github/GPX6/prep_structures/humansec/N3K.pdb.
PyMOL>select chain X and resi 4
 Selector: selection "sele" defined with 61 atoms.
Selected!
Mutation applied at residue 4: R -> SER. Saved to /home/hp/nayanika/github/GPX6/prep_structures/humansec/R4S.pdb.
PyMOL>select chain X and resi 16
Selected!
 Selector: selection "sele" defined with 47 atoms.
Mutation applied at residue 16: I -> VAL. Saved to /home/hp/nayanika/github/GPX6/prep_structures/humansec/I16V.pdb.
PyMOL>select chain X and resi 22
Selected!
 Selector: selection "sele" defined with 48 atoms.
Mutation applied at residue 22: L -> ASN. Saved to /home/hp/nayanika/github/GPX6/prep_structures/humansec/L22N.pdb.
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/hp/anaconda3/envs/openMM/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_77724/522560991.py", line 96, in <module>
    mutate_residues(pdb_file_path, mutations_list)
  File "/tmp/ipykernel_77724/522560991.py", line 59, in mutate_residues
    cmd.wizard("mutagenesis")
  File "/home/hp/anaconda3/envs/openMM/lib/python3.8/site-packages/pymol/wizarding.py", line 90, in wizard
    r = _wizard(name,arg,kwd,0,_self=_self)
  File "/home/hp/anaconda3/envs/openMM/lib/python3.8/site-packages/pymol/wizarding.py", line 46, in _wizard
    wiz = getattr(mod_obj,oname)(*arg, **kwd)
  File "/home/hp/anaconda3/envs/openMM/lib/python3.8/site-packages/pymol/wizard/mutagenesis.py", line 60, in __init__
    self.load_library()
  File "/home/hp/anaconda3/envs/openMM/lib/python3.8/site-packages/pymol/wizard/mutagenesis.py", line 223, in load_librar